# Microbleed FAKE Label Generator

The Purpose of this notebook is purely to refine the Australia Public Share Data. Creating a nifty label using coordinate system ignoring shape and size (will create a class 1 for specified coordinate and += voxels surrounding the corrdinates). 

This leverages provided coordinates system of known cerebral microbleeds and generates a mimiced microbleed in each location. 

This is used for evaluation(analyse actual predictions to these generated template labels to check recall and precision) and object detection purposes
  

## Setup and Imports


In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation

Mounted at /content/drive
/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation


In [ ]:
!pip install -r Microbleed_Segmentation/requirements.txt

     |████████████████████████████████| 8.3 MB 6.8 MB/s 
     |████████████████████████████████| 492 kB 42.3 MB/s 
     |████████████████████████████████| 56 kB 3.6 MB/s 
     |████████████████████████████████| 428 kB 40.5 MB/s 
     |████████████████████████████████| 392 kB 46.5 MB/s 
     |████████████████████████████████| 68 kB 6.6 MB/s 
     |████████████████████████████████| 112 kB 46.9 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled 

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/Microbleed_Segmentation/src')

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AddChanneld,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    RandSpatialCrop,
    Spacingd,
    ToTensord,
    NormalizeIntensityd,
    RandAffined,
    ResizeWithPadOrCropd,
    RandWeightedCropd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandZoomd,
    RandSpatialCropd,
    SpatialCropd
)
from sampling import RandUniformSamplingCropd
from monai.networks.nets import UNet, BasicUNet
from monai.networks.layers import Norm
from monai.metrics import compute_meandice
from monai.losses import DiceLoss, DiceCELoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset
from monai.data.utils import pad_list_data_collate
from monai.config import print_config
from monai.apps import download_and_extract
import numpy as np
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import monai  

import math

import pandas as pd
import nibabel as nb

from Transforms import *

from UNet import UNet

from typing import Any, Callable, Dict, Hashable, List, Mapping

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

## Helper Functions

In [ ]:
# Helper functions
class DataDict():
    Image = 'image'
    Label = 'label'
    FileName = 'filename'
    ID = 'id'
    VersionId = 'version_id'
    ScanTimeVersion = 'scan_time_version'
    X = 'x'
    Y = 'y'
    Z = 'z'

class SpatialCropd(SpatialCropd):
    def __call__(self, data: Mapping[Hashable, np.ndarray]) -> Dict[Hashable, np.ndarray]:
        d = dict(data)
        # print(len(d))
        for key in self.key_iterator(d):
            self.push_transform(d, key)
            print(f"key is {key}")
            print(f"shape is {d[key].shape}")
            print(f"self cropper coord {self.cropper.roi_center}")
            d[key] = self.cropper(d[key])
            print(f"shape after is {d[key].shape}")
        return d

def get_swi_subject_id(file_name, ext='.nii.gz'):
    return int(file_name.split("EPAD")[-1].replace(ext, "").replace("_swi", "").replace("_cmb_seg", ""))

def get_subject_id_from_image_label(path1, path2):
    path1_id = get_swi_subject_id(path1)
    path2_id = get_swi_subject_id(path2)

    if not path1_id == path2_id:
        raise ValueError("Subject mismatch")

    return path1_id


# Convert torch tensor to numpy array
def numpy_from_tensor(x):
  return x.detach().cpu().numpy()

# Predefined checkpoint dictionary
class CheckpointDict():
    EPOCH = 'epoch'
    MODEL_STATE_DICT = 'model_state_dict'
    OPTIMIZER_STATE_DICT = 'optimizer_state_dict'
    LOSS = 'loss'
    LOSSES = 'losses'
    VALIDATION_LOSSES = 'val_losses'


def display_losses(checkpoint, title='Network Losses', x_label='Epoch', y_label='Loss', combine_train_val=True):
    train_losses, val_losses = get_checkpoint_losses(checkpoint)

    val_losses = dict_tensor_to_cpu(val_losses)
    val_tensors = dict_tensor_to_value(val_losses)

    training_losses = sorted(train_losses.items())  # sorted by key, return a list of tuples
    x, y = zip(*training_losses)  # unpack a list of pairs into two tuples
    validation_losses = sorted(val_tensors.items())
    x2, y2 = zip(*validation_losses)  # unpack a list of pairs into two tuples

    if combine_train_val:
        plt.title(title)
        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.plot(x, y, label="Training")
        plt.plot(x2, y2, label="Validation")
        plt.legend()
        plt.show()
    else:
        fig, axs = plt.subplots(2, 1, constrained_layout=True)
        fig.suptitle(title, fontsize=16)

        axs[0].plot(x, y)
        axs[0].set_title('Training Losses')
        axs[0].set_xlabel(x_label)
        axs[0].set_ylabel(y_label)

        axs[1].plot(x2, y2, color='r')
        axs[1].set_title('Validation Losses')
        axs[1].set_xlabel(x_label)
        axs[1].set_ylabel(y_label)

        plt.show()

def get_checkpoint_losses(checkpoint):
    losses = checkpoint[CheckpointDict.LOSSES]
    val_losses = checkpoint[CheckpointDict.VALIDATION_LOSSES]
    val_losses = dict_tensor_to_cpu(val_losses)
    val_tensors = dict_tensor_to_value(val_losses)
    return losses, val_losses

def get_checkpoint_model_state_dict(checkpoint):
    return checkpoint[CheckpointDict.MODEL_STATE_DICT]


def dict_tensor_to_cpu(dict_tensors):
    for key in dict_tensors.keys():
        dict_tensors[key] = tensor_to_cpu(dict_tensors[key])

    return dict_tensors


def tensor_to_cpu(x):
    return x.detach().cpu()


def dict_tensor_to_value(dicts):
    if next(iter(dicts)) is not type(torch.tensor):
        return dicts

    for keys in dicts:
        dicts[keys] = dicts[keys].item()

    return dicts


# Get dictionarys from a list_dict filtered by a spcific key value
def get_dicts_from_dicts(dicts, key, value):
    x = [item for item in dicts if item[key] == value]
    return x


post_pred = AsDiscrete(n_classes=2, threshold_values=True)
post_label = AsDiscrete(to_onehot=True, n_classes=2)

## Coordinates Dataframe

### Real Microbleed DataFrame

In [ ]:
# Get Coordinates for labels
coord_file_path_real = "/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/rCMBInformationInfo.xlsx"


coordinate_info_real_microbleed_df = pd.read_excel(coord_file_path_real)
N = 52
coordinate_info_real_microbleed_df  = coordinate_info_real_microbleed_df.iloc[: , :N]
# coordinate_info_real_microbleed_df

In [ ]:
coordinate_info_real_microbleed_df.rename(columns={"Location CMB # 1": "11-1", "Unnamed: 2": "11-2", "Unnamed: 3": "11-3",
                   "Location CMB # 2": "12-1", "Unnamed: 5": "12-2", "Unnamed: 6": "12-3",
                   "Location CMB # 3": "13-1", "Unnamed: 8": "13-2", "Unnamed: 9": "13-3",
                   "Location CMB # 4": "14-1", "Unnamed: 11": "14-2", "Unnamed: 12": "14-3",
                   "Location CMB # 5": "15-1", "Unnamed: 14": "15-2", "Unnamed: 15": "15-3",
                   "Location CMB # 6": "16-1", "Unnamed: 17": "16-2", "Unnamed: 18": "16-3",
                   "Location CMB # 7": "17-1", "Unnamed: 20": "17-2", "Unnamed: 21": "17-3",
                   "Location CMB # 8": "18-1", "Unnamed: 23": "18-2", "Unnamed: 24": "18-3",
                   "Location CMB # 9": "19-1", "Unnamed: 26": "19-2", "Unnamed: 27": "19-3",
                   "Location CMB # 10": "20-1", "Unnamed: 29": "20-2", "Unnamed: 30": "20-3",
                   "Location CMB # 11": "21-1", "Unnamed: 32": "21-2", "Unnamed: 33": "21-3",
                   "Location CMB # 12": "22-1", "Unnamed: 35": "22-2", "Unnamed: 36": "22-3",
                   "Location CMB # 13": "23-1", "Unnamed: 38": "23-2", "Unnamed: 39": "23-3",
                   "Location CMB # 14": "24-1", "Unnamed: 41": "24-2", "Unnamed: 42": "24-3",
                   "Location CMB # 15": "25-1", "Unnamed: 44": "25-2", "Unnamed: 45": "25-3",
                   "Location CMB # 16": "26-1", "Unnamed: 47": "26-2", "Unnamed: 48": "26-3",
                   "Location CMB # 17": "27-1", "Unnamed: 50": "27-2", "Unnamed: 51": "27-3",
                   }, inplace=True)

coordinate_info_real_microbleed_df

,NIFTI File Name,11-1,11-2,11-3,12-1,12-2,12-3,13-1,13-2,13-3,14-1,14-2,14-3,15-1,15-2,15-3,16-1,16-2,16-3,17-1,17-2,17-3,18-1,18-2,18-3,19-1,19-2,19-3,20-1,20-2,20-3,21-1,21-2,21-3,22-1,22-2,22-3,23-1,23-2,23-3,24-1,24-2,24-3,25-1,25-2,25-3,26-1,26-2,26-3,27-1,27-2,27-3
0,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,122_T2_MRI_SWI_BFC_50mm_HM.nii.gz,51,167,26,65,171,13,132,102,39,131,123,43,126,129,45,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,218_T0_MRI_SWI_BFC_50mm_HM.nii.gz,74,175,13,143,150,56,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,218_T2_MRI_SWI_BFC_50mm_HM.nii.gz,75,177,14,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,218_T3_MRI_SWI_BFC_50mm_HM.nii.gz,71,178,16,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,219_T0_MRI_SWI_BFC_50mm_HM.nii.gz,119,112,42,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6,221_T2_MRI_SWI_BFC_50mm_HM.nii.gz,137,143,22,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,222_T0_MRI_SWI_BFC_50mm_HM.nii.gz,149,164,37,89,86,60,97,62,58,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8,222_T1_MRI_SWI_BFC_50mm_HM.nii.gz,67,219,32,93,76,64,86,100,65,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9,238_T0_MRI_SWI_BFC_50mm_HM.nii.gz,65,176,6,122,173,7,128,204,31,112,82,49,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


### Synthetic Microbleed DataFrame

In [ ]:
# Get Coordinates for labels - Synthetic
coord_file_path_synthetic = "/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMBInformationInfo.xlsx"


coordinate_info_synthetic_microbleed_df = pd.read_excel(coord_file_path_synthetic)
N = 31
coordinate_info_synthetic_microbleed_df  = coordinate_info_synthetic_microbleed_df.iloc[: , :N]
coordinate_info_synthetic_microbleed_df

,NIFTI File Name,Location CMB #1,Unnamed: 2,Unnamed: 3,Location CMB #2,Unnamed: 5,Unnamed: 6,Location CMB #3,Unnamed: 8,Unnamed: 9,Location CMB #4,Unnamed: 11,Unnamed: 12,Location CMB #5,Unnamed: 14,Unnamed: 15,Location CMB #6,Unnamed: 17,Unnamed: 18,Location CMB #7,Unnamed: 20,Unnamed: 21,Location CMB #8,Unnamed: 23,Unnamed: 24,Location CMB #9,Unnamed: 26,Unnamed: 27,Location CMB #10,Unnamed: 29,Unnamed: 30
0,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz,103,64,46,35,151,46,46,130,30,34,161,40,38,116,27,57,202,28,56,141,63,100,80,63,97,161,64,121,183,20
1,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz,140,116,28,118,196,31,97,161,64,53,172,11,60,190,23,69,72,42,98,145,19,64,154,23,144,166,54,147,144,48
2,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz,83,87,65,123,87,36,58,181,60,93,114,64,39,108,32,49,86,44,125,171,14,149,153,41,83,87,65,59,198,28
3,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V4.nii.gz,42,153,61,123,87,36,56,141,63,72,174,71,57,202,28,73,188,18,154,156,40,82,196,24,97,137,64,123,87,36
4,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V5.nii.gz,61,143,35,89,207,33,93,129,68,77,200,57,112,178,67,143,196,48,127,175,8,151,180,42,42,153,61,49,86,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V6.nii.gz,129,109,56,47,186,27,98,61,44,88,135,24,114,197,16,59,201,22,130,169,52,32,139,34,111,108,33,72,182,12
566,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V7.nii.gz,106,129,29,49,124,59,75,201,54,44,173,24,31,131,37,130,169,52,47,202,32,57,156,25,102,143,20,79,155,14
567,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V8.nii.gz,44,158,57,49,124,59,128,175,25,123,135,32,62,106,37,59,81,46,145,163,41,47,148,32,145,166,44,62,150,24
568,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V9.nii.gz,50,197,34,97,199,23,111,108,33,47,202,32,120,157,29,123,110,59,80,131,66,106,129,29,87,168,11,126,160,17


In [ ]:
coordinate_info_synthetic_microbleed_df.rename(columns={"Location CMB #1": "1-1", "Unnamed: 2": "1-2", "Unnamed: 3": "1-3",
                   "Location CMB #2": "2-1", "Unnamed: 5": "2-2", "Unnamed: 6": "2-3",
                   "Location CMB #3": "3-1", "Unnamed: 8": "3-2", "Unnamed: 9": "3-3",
                   "Location CMB #4": "4-1", "Unnamed: 11": "4-2", "Unnamed: 12": "4-3",
                   "Location CMB #5": "5-1", "Unnamed: 14": "5-2", "Unnamed: 15": "5-3",
                   "Location CMB #6": "6-1", "Unnamed: 17": "6-2", "Unnamed: 18": "6-3",
                   "Location CMB #7": "7-1", "Unnamed: 20": "7-2", "Unnamed: 21": "7-3",
                   "Location CMB #8": "8-1", "Unnamed: 23": "8-2", "Unnamed: 24": "8-3",
                   "Location CMB #9": "9-1", "Unnamed: 26": "9-2", "Unnamed: 27": "9-3",
                   "Location CMB #10": "10-1", "Unnamed: 29": "10-2", "Unnamed: 30": "10-3"
                   }, inplace=True)

coordinate_info_synthetic_microbleed_df

,NIFTI File Name,1-1,1-2,1-3,2-1,2-2,2-3,3-1,3-2,3-3,4-1,4-2,4-3,5-1,5-2,5-3,6-1,6-2,6-3,7-1,7-2,7-3,8-1,8-2,8-3,9-1,9-2,9-3,10-1,10-2,10-3
0,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz,103,64,46,35,151,46,46,130,30,34,161,40,38,116,27,57,202,28,56,141,63,100,80,63,97,161,64,121,183,20
1,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz,140,116,28,118,196,31,97,161,64,53,172,11,60,190,23,69,72,42,98,145,19,64,154,23,144,166,54,147,144,48
2,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz,83,87,65,123,87,36,58,181,60,93,114,64,39,108,32,49,86,44,125,171,14,149,153,41,83,87,65,59,198,28
3,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V4.nii.gz,42,153,61,123,87,36,56,141,63,72,174,71,57,202,28,73,188,18,154,156,40,82,196,24,97,137,64,123,87,36
4,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V5.nii.gz,61,143,35,89,207,33,93,129,68,77,200,57,112,178,67,143,196,48,127,175,8,151,180,42,42,153,61,49,86,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V6.nii.gz,129,109,56,47,186,27,98,61,44,88,135,24,114,197,16,59,201,22,130,169,52,32,139,34,111,108,33,72,182,12
566,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V7.nii.gz,106,129,29,49,124,59,75,201,54,44,173,24,31,131,37,130,169,52,47,202,32,57,156,25,102,143,20,79,155,14
567,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V8.nii.gz,44,158,57,49,124,59,128,175,25,123,135,32,62,106,37,59,81,46,145,163,41,47,148,32,145,166,44,62,150,24
568,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V9.nii.gz,50,197,34,97,199,23,111,108,33,47,202,32,120,157,29,123,110,59,80,131,66,106,129,29,87,168,11,126,160,17


### Combine into one dataframe

In [ ]:
# Create matching column names to merge (real and synthetic titled slightly differently)
repeat_count = 20
# Duplicate 20 times over row to ensure all versions (check synthetic) are included
coordinate_info_real_microbleed_formatted_df = pd.concat([coordinate_info_real_microbleed_df]*repeat_count, ignore_index=True)

# Group common rows together and reset index
coordinate_info_real_microbleed_formatted_df.sort_values(by='NIFTI File Name', inplace=True)
coordinate_info_real_microbleed_formatted_df.reset_index(inplace=True)

coordinate_info_real_microbleed_formatted_df

,index,NIFTI File Name,11-1,11-2,11-3,12-1,12-2,12-3,13-1,13-2,13-3,14-1,14-2,14-3,15-1,15-2,15-3,16-1,16-2,16-3,17-1,17-2,17-3,18-1,18-2,18-3,19-1,19-2,19-3,20-1,20-2,20-3,21-1,21-2,21-3,22-1,22-2,22-3,23-1,23-2,23-3,24-1,24-2,24-3,25-1,25-2,25-3,26-1,26-2,26-3,27-1,27-2,27-3
0,0,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,57,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,969,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,912,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,114,122_T1_MRI_SWI_BFC_50mm_HM.nii.gz,64,175,12,125,134,44,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,900,3_T2_MRI_SWI_BFC_50mm_HM.nii.gz,77,173,55,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1136,957,3_T2_MRI_SWI_BFC_50mm_HM.nii.gz,77,173,55,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1137,1014,3_T2_MRI_SWI_BFC_50mm_HM.nii.gz,77,173,55,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1138,330,3_T2_MRI_SWI_BFC_50mm_HM.nii.gz,77,173,55,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
# Rename header name to match synthetic data type
for i, row in coordinate_info_real_microbleed_formatted_df.iterrows():
    version = i % repeat_count + 1

    to_replace = row['NIFTI File Name']
    to_replace = to_replace.replace("_HM.nii.gz", f"_HM_rsCMB_V{str(version)}.nii.gz")

    coordinate_info_real_microbleed_formatted_df.at[i, 'NIFTI File Name'] = to_replace

In [ ]:
# Merge synthetic and real data together into one pandas
coordinate_info_df = coordinate_info_synthetic_microbleed_df.merge(coordinate_info_real_microbleed_formatted_df, how = 'inner', on='NIFTI File Name').drop_duplicates()
coordinate_info_df = coordinate_info_df.drop(['index'], axis=1)
coordinate_info_df

,NIFTI File Name,1-1,1-2,1-3,2-1,2-2,2-3,3-1,3-2,3-3,4-1,4-2,4-3,5-1,5-2,5-3,6-1,6-2,6-3,7-1,7-2,7-3,8-1,8-2,8-3,9-1,9-2,9-3,10-1,10-2,10-3,11-1,11-2,11-3,12-1,12-2,12-3,13-1,13-2,13-3,...,14-3,15-1,15-2,15-3,16-1,16-2,16-3,17-1,17-2,17-3,18-1,18-2,18-3,19-1,19-2,19-3,20-1,20-2,20-3,21-1,21-2,21-3,22-1,22-2,22-3,23-1,23-2,23-3,24-1,24-2,24-3,25-1,25-2,25-3,26-1,26-2,26-3,27-1,27-2,27-3
0,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz,103,64,46,35,151,46,46,130,30,34,161,40,38,116,27,57,202,28,56,141,63,100,80,63,97,161,64,121,183,20,64,175,12,125,134,44,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz,140,116,28,118,196,31,97,161,64,53,172,11,60,190,23,69,72,42,98,145,19,64,154,23,144,166,54,147,144,48,64,175,12,125,134,44,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz,83,87,65,123,87,36,58,181,60,93,114,64,39,108,32,49,86,44,125,171,14,149,153,41,83,87,65,59,198,28,64,175,12,125,134,44,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V4.nii.gz,42,153,61,123,87,36,56,141,63,72,174,71,57,202,28,73,188,18,154,156,40,82,196,24,97,137,64,123,87,36,64,175,12,125,134,44,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V5.nii.gz,61,143,35,89,207,33,93,129,68,77,200,57,112,178,67,143,196,48,127,175,8,151,180,42,42,153,61,49,86,44,64,175,12,125,134,44,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V6.nii.gz,129,109,56,47,186,27,98,61,44,88,135,24,114,197,16,59,201,22,130,169,52,32,139,34,111,108,33,72,182,12,126,203,43,98,225,34,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
566,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V7.nii.gz,106,129,29,49,124,59,75,201,54,44,173,24,31,131,37,130,169,52,47,202,32,57,156,25,102,143,20,79,155,14,126,203,43,98,225,34,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
567,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V8.nii.gz,44,158,57,49,124,59,128,175,25,123,135,32,62,106,37,59,81,46,145,163,41,47,148,32,145,166,44,62,150,24,126,203,43,98,225,34,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
568,316_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V9.nii.gz,50,197,34,97,199,23,111,108,33,47,202,32,120,157,29,123,110,59,80,131,66,106,129,29,87,168,11,126,160,17,126,203,43,98,225,34,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## Label Generator

In [ ]:
# Build list dictionary containing information about the dataframe above and coordinate systems
def get_coord_dicts(dataframe, data_paths, filename_column="NIFTI File Name", microbleeds_per_subj=27):
    """
      dataframe: dataframe containining column with filename and columns specifying coordinates for each microbleed
    """
    list_data_dict = []

    for data_path in data_paths:
        target_df_row = dataframe.loc[dataframe['NIFTI File Name'] == data_path[DataDict.FileName]]
        
        for i in range(microbleeds_per_subj):
            x = target_df_row.iloc[0][f'{i+1}-1']
            y = target_df_row.iloc[0][f'{i+1}-2']
            z = target_df_row.iloc[0][f'{i+1}-3']

            if isinstance(x, str) or isinstance(y, str) or isinstance(z, str):
                continue

            filename = data_path[DataDict.FileName]
            # extract ids from filename: 218_T0_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz
            subject_id = int(filename.split('_')[0])
            version_id = int(filename.split('_')[-1].replace('.nii.gz', '').replace('V', ''))
            scan_t_version_id = int(filename.split('_')[1].replace('T', ''))

            dict_to_append = {DataDict.FileName: data_path[DataDict.FileName], 
                              DataDict.Image: data_path[DataDict.Image], 
                              DataDict.ID: subject_id,
                              DataDict.VersionId: version_id,
                              DataDict.ScanTimeVersion: scan_t_version_id,
                              DataDict.X: target_df_row.iloc[0][f'{i+1}-1'],
                              DataDict.Y: target_df_row.iloc[0][f'{i+1}-2'],
                              DataDict.Z: target_df_row.iloc[0][f'{i+1}-3']}

            list_data_dict.append(dict_to_append)

    return list_data_dict

In [ ]:
# Synthetic microbleed training data
test_data_path = f"/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/"

test_data_paths = []

# Append list dict of training data into a list
for index, row in coordinate_info_df.iterrows():
    filename = row['NIFTI File Name']
    image = test_data_path + row['NIFTI File Name']

    id = int(filename.split('_')[0])
    test_data_paths.append({DataDict.FileName: filename, DataDict.Image: image})

In [ ]:
test_data_paths

[{'filename': '122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz',
  'image': '/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V1.nii.gz'},
 {'filename': '122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz',
  'image': '/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz'},
 {'filename': '122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz',
  'image': '/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V3.nii.gz'},
 {'filename': '122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V4.nii.gz',
  'image': '/

In [ ]:
included_filenames = [item[DataDict.FileName] for item in test_data_paths]

In [ ]:
# List dicts per microbleed containing file path and microbleed coordinates
test_data_dicts = get_coord_dicts(coordinate_info_df, test_data_paths)

In [ ]:
test_data_dicts[20]

{'filename': '122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz',
 'id': 122,
 'image': '/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2.nii.gz',
 'scan_time_version': 1,
 'version_id': 2,
 'x': 144,
 'y': 166,
 'z': 54}

In [ ]:
len(test_data_dicts)

7160

In [ ]:
# Declare test transforms
test_transforms = Compose(
    [
        LoadImaged(keys=[DataDict.Image]),
        AddChanneld(keys=[DataDict.Image]),
        Spacingd(keys=[DataDict.Image], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear")),
        Orientationd(keys=[DataDict.Image], axcodes="RAS"),
        NormalizeIntensityd(keys=[DataDict.Image]),
        ToTensord(keys=[DataDict.Image]),
    ]
)

In [ ]:
# Final Inference code
generate_labels = True

target_output_folder = f"/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/Microbleed_Segmentation/GeneratedLabels/FakeGeneratedLabels/"
fake_microbleed_radius = 0
width = 1 + (2 * fake_microbleed_radius)

if os.path.exists(target_output_folder) == False:
    os.mkdir(target_output_folder)

fake_microbleed = torch.ones((width, width, width))

if generate_labels: 
    # Iterate over all inluded filenames and infer
    for filename_idx, included_filename in enumerate(included_filenames):
        print(f"Analysing subject {filename_idx+1}/{len(included_filenames)}")
        same_subject_microbleeds_dict = get_dicts_from_dicts(test_data_dicts, DataDict.FileName, included_filename)

        # Fileimage path should be the same so get first
        nibabel_file = nb.load(same_subject_microbleeds_dict[0][DataDict.Image])
        swi_image = nibabel_file.get_fdata()

        # Initialise data
        current_dataset = Dataset(data=[same_subject_microbleeds_dict[0]], transform=test_transforms)
        current_data_loader = DataLoader(current_dataset, batch_size=1, num_workers=0)

        # Initialise image and label template
        current_data = next(iter(current_data_loader))
        current_subject_image = current_data[DataDict.Image].squeeze_().squeeze_()

        # Get Subject Id
        subject_id = int(current_data[DataDict.ID])
        version_id = int(current_data[DataDict.VersionId])
        scan_time_version_id = int(current_data[DataDict.ScanTimeVersion])

        # print(f"nib shape: {swi_image.shape}, current_sub shape: {current_subject_image.shape}")


        # Create zero tensor for label template
        # test_label = torch.zeros_like(current_subject_image)
        test_label = torch.zeros_like(torch.tensor(swi_image))
        
        # Iterate over dictionary to infer mirobleed
        for idx, microbleed_dict in enumerate(same_subject_microbleeds_dict):
            # Get current microbleed coordinate _ NOTE coordinates start from 1 and images start from 0
            x, y, z = microbleed_dict[DataDict.X]-1, microbleed_dict[DataDict.Y]-1, microbleed_dict[DataDict.Z]-1

            # Assign coordinate and specified surrounding area with fake microbleed template
            test_label[x-fake_microbleed_radius:x+fake_microbleed_radius+1, y-fake_microbleed_radius:y+fake_microbleed_radius+1, z-fake_microbleed_radius:z+fake_microbleed_radius+1] = fake_microbleed


        numpy_output = np.array(test_label)
        # Must be converted as numpy.single due to formatting issues
        numpy_output = numpy_output.astype(np.single)

        # Keep and match affine transformation to original image
        nb_image_to_save = nb.Nifti1Image(numpy_output, nibabel_file.affine)
        # nb_image_to_save = nb.Nifti1Image(numpy_output, np.eye(4))
        
        subject_dir_path = os.path.join(target_output_folder, f'{subject_id}_T{scan_time_version_id}_V{version_id}/')

        if os.path.exists(subject_dir_path) == False:
            os.mkdir(subject_dir_path)

        # Save generated image to specified location
        nb.save(nb_image_to_save, f'{subject_dir_path}/fake_label_{included_filename}')
        print("\n")
    
print("DONE")

Analysing subject 1/570


Analysing subject 2/570


Analysing subject 3/570


Analysing subject 4/570


Analysing subject 5/570


Analysing subject 6/570


Analysing subject 7/570


Analysing subject 8/570


Analysing subject 9/570


Analysing subject 10/570


Analysing subject 11/570


Analysing subject 12/570


Analysing subject 13/570


Analysing subject 14/570


Analysing subject 15/570


Analysing subject 16/570


Analysing subject 17/570


Analysing subject 18/570


Analysing subject 19/570


Analysing subject 20/570


Analysing subject 21/570


Analysing subject 22/570


Analysing subject 23/570


Analysing subject 24/570


Analysing subject 25/570


Analysing subject 26/570


Analysing subject 27/570


Analysing subject 28/570


Analysing subject 29/570


Analysing subject 30/570


Analysing subject 31/570


Analysing subject 32/570


Analysing subject 33/570


Analysing subject 34/570


Analysing subject 35/570


Analysing subject 36/570


Analysing subject 37/570


Analysing 

In [ ]:
# Final Inference code
generate_labels = True

target_output_folder = f"/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/Microbleed_Segmentation/GeneratedLabels/FakeGeneratedLabels_3Width/"
fake_microbleed_radius = 1
width = 1 + (2 * fake_microbleed_radius)

if os.path.exists(target_output_folder) == False:
    os.mkdir(target_output_folder)

fake_microbleed = torch.ones((width, width, width))

if generate_labels: 
    # Iterate over all inluded filenames and infer
    for filename_idx, included_filename in enumerate(included_filenames):
        print(f"Analysing subject {filename_idx+1}/{len(included_filenames)}")
        same_subject_microbleeds_dict = get_dicts_from_dicts(test_data_dicts, DataDict.FileName, included_filename)

        # Fileimage path should be the same so get first
        nibabel_file = nb.load(same_subject_microbleeds_dict[0][DataDict.Image])
        swi_image = nibabel_file.get_fdata()

        # Initialise data
        current_dataset = Dataset(data=[same_subject_microbleeds_dict[0]], transform=test_transforms)
        current_data_loader = DataLoader(current_dataset, batch_size=1, num_workers=0)

        # Initialise image and label template
        current_data = next(iter(current_data_loader))
        current_subject_image = current_data[DataDict.Image].squeeze_().squeeze_()

        # Get Subject Id
        subject_id = int(current_data[DataDict.ID])
        version_id = int(current_data[DataDict.VersionId])
        scan_time_version_id = int(current_data[DataDict.ScanTimeVersion])

        # print(f"nib shape: {swi_image.shape}, current_sub shape: {current_subject_image.shape}")


        # Create zero tensor for label template
        # test_label = torch.zeros_like(current_subject_image)
        test_label = torch.zeros_like(torch.tensor(swi_image))
        
        # Iterate over dictionary to infer mirobleed
        for idx, microbleed_dict in enumerate(same_subject_microbleeds_dict):
            # Get current microbleed coordinate _ NOTE coordinates start from 1 and images start from 0
            x, y, z = microbleed_dict[DataDict.X]-1, microbleed_dict[DataDict.Y]-1, microbleed_dict[DataDict.Z]-1

            # Assign coordinate and specified surrounding area with fake microbleed template
            test_label[x-fake_microbleed_radius:x+fake_microbleed_radius+1, y-fake_microbleed_radius:y+fake_microbleed_radius+1, z-fake_microbleed_radius:z+fake_microbleed_radius+1] = fake_microbleed


        numpy_output = np.array(test_label)
        # Must be converted as numpy.single due to formatting issues
        numpy_output = numpy_output.astype(np.single)

        # Keep and match affine transformation to original image
        nb_image_to_save = nb.Nifti1Image(numpy_output, nibabel_file.affine)
        # nb_image_to_save = nb.Nifti1Image(numpy_output, np.eye(4))
        
        subject_dir_path = os.path.join(target_output_folder, f'{subject_id}_T{scan_time_version_id}_V{version_id}/')

        if os.path.exists(subject_dir_path) == False:
            os.mkdir(subject_dir_path)

        nb.save(nb_image_to_save, f'{subject_dir_path}/fake_label_{included_filename}')
        print("\n")
    
print("DONE")

Analysing subject 1/570


Analysing subject 2/570


Analysing subject 3/570


Analysing subject 4/570


Analysing subject 5/570


Analysing subject 6/570


Analysing subject 7/570


Analysing subject 8/570


Analysing subject 9/570


Analysing subject 10/570


Analysing subject 11/570


Analysing subject 12/570


Analysing subject 13/570


Analysing subject 14/570


Analysing subject 15/570


Analysing subject 16/570


Analysing subject 17/570


Analysing subject 18/570


Analysing subject 19/570


Analysing subject 20/570


Analysing subject 21/570


Analysing subject 22/570


Analysing subject 23/570


Analysing subject 24/570


Analysing subject 25/570


Analysing subject 26/570


Analysing subject 27/570


Analysing subject 28/570


Analysing subject 29/570


Analysing subject 30/570


Analysing subject 31/570


Analysing subject 32/570


Analysing subject 33/570


Analysing subject 34/570


Analysing subject 35/570


Analysing subject 36/570


Analysing subject 37/570


Analysing 

In [ ]:
# Final Inference code
generate_labels = True

target_output_folder = f"/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/Microbleed_Segmentation/GeneratedLabels/FakeGeneratedLabels_5Width/"
fake_microbleed_radius = 2
width = 1 + (2 * fake_microbleed_radius)

if os.path.exists(target_output_folder) == False:
    os.mkdir(target_output_folder)

fake_microbleed = torch.ones((width, width, width))

if generate_labels: 
    # Iterate over all inluded filenames and infer
    for filename_idx, included_filename in enumerate(included_filenames):
        print(f"Analysing subject {filename_idx+1}/{len(included_filenames)}")
        same_subject_microbleeds_dict = get_dicts_from_dicts(test_data_dicts, DataDict.FileName, included_filename)

        # Fileimage path should be the same so get first
        nibabel_file = nb.load(same_subject_microbleeds_dict[0][DataDict.Image])
        swi_image = nibabel_file.get_fdata()

        # Initialise data
        current_dataset = Dataset(data=[same_subject_microbleeds_dict[0]], transform=test_transforms)
        current_data_loader = DataLoader(current_dataset, batch_size=1, num_workers=0)

        # Initialise image and label template
        current_data = next(iter(current_data_loader))
        current_subject_image = current_data[DataDict.Image].squeeze_().squeeze_()

        # Get Subject Id
        subject_id = int(current_data[DataDict.ID])
        version_id = int(current_data[DataDict.VersionId])
        scan_time_version_id = int(current_data[DataDict.ScanTimeVersion])

        # print(f"nib shape: {swi_image.shape}, current_sub shape: {current_subject_image.shape}")


        # Create zero tensor for label template
        # test_label = torch.zeros_like(current_subject_image)
        test_label = torch.zeros_like(torch.tensor(swi_image))
        
        # Iterate over dictionary to infer mirobleed
        for idx, microbleed_dict in enumerate(same_subject_microbleeds_dict):
            # Get current microbleed coordinate _ NOTE coordinates start from 1 and images start from 0
            x, y, z = microbleed_dict[DataDict.X]-1, microbleed_dict[DataDict.Y]-1, microbleed_dict[DataDict.Z]-1

            # Assign coordinate and specified surrounding area with fake microbleed template
            test_label[x-fake_microbleed_radius:x+fake_microbleed_radius+1, y-fake_microbleed_radius:y+fake_microbleed_radius+1, z-fake_microbleed_radius:z+fake_microbleed_radius+1] = fake_microbleed


        numpy_output = np.array(test_label)
        # Must be converted as numpy.single due to formatting issues
        numpy_output = numpy_output.astype(np.single)

        # Keep and match affine transformation to original image
        nb_image_to_save = nb.Nifti1Image(numpy_output, nibabel_file.affine)
        # nb_image_to_save = nb.Nifti1Image(numpy_output, np.eye(4))
        
        subject_dir_path = os.path.join(target_output_folder, f'{subject_id}_T{scan_time_version_id}_V{version_id}/')

        if os.path.exists(subject_dir_path) == False:
            os.mkdir(subject_dir_path)

        nb.save(nb_image_to_save, f'{subject_dir_path}/fake_label_{included_filename}')
        print("\n")
    
print("DONE")

Analysing subject 1/570


Analysing subject 2/570


Analysing subject 3/570


Analysing subject 4/570


Analysing subject 5/570


Analysing subject 6/570


Analysing subject 7/570


Analysing subject 8/570


Analysing subject 9/570


Analysing subject 10/570


Analysing subject 11/570


Analysing subject 12/570


Analysing subject 13/570


Analysing subject 14/570


Analysing subject 15/570


Analysing subject 16/570


Analysing subject 17/570


Analysing subject 18/570


Analysing subject 19/570


Analysing subject 20/570


Analysing subject 21/570


Analysing subject 22/570


Analysing subject 23/570


Analysing subject 24/570


Analysing subject 25/570


Analysing subject 26/570


Analysing subject 27/570


Analysing subject 28/570


Analysing subject 29/570


Analysing subject 30/570


Analysing subject 31/570


Analysing subject 32/570


Analysing subject 33/570


Analysing subject 34/570


Analysing subject 35/570


Analysing subject 36/570


Analysing subject 37/570


Analysing 

In [ ]:
predicted_directories = glob.glob(f"{target_output_folder}/*/")
len(predicted_directories) # should be same as analysed i.e 570

## Testing sucessful predictions (False Negatives) DELETE FOR FAKE GENERATED LABELS


In [ ]:
from EvalCMBSeg import *

segoutPath = "/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/PredictedLabels/TestEval/Predictions"
labelPath= "/content/drive/MyDrive/Brain Health In COVID 19/Microbleed_Segmentation/EPAD_WMD/SyntheticCMBPaperData/webdav-bm.data.csiro.au/dap_prd_000050304v001/PublicDataShare_2020/sCMB_DefiniteSubject/PredictedLabels/TestEval/Labels"

labelPrefix = "label_"
imagePrefix = "seg_out_"

results =  get_metrics(using_folds=True)        

print(results)

## IGNORE BELOW CODE USED FOR TESTING

In [ ]:
test_tensor = torch.tensor([[0, 0, 1], [0, 0, 0]])
test_tensor2 = torch.tensor([[0, 0, 0], [0, 0, 0]])
print(1 in test_tensor, 1 in test_tensor2)

In [ ]:
predicted_labels_list = glob.glob(f"{target_output_folder}*.nii.gz")
# predicted_labels_list = os.listdir(target_output_folder)
os.path.basename(predicted_labels_list[0])

In [ ]:
# predicted_labels_list = glob.glob(f"{target_output_folder}*.nii.gz")
predicted_labels_list = os.listdir(target_output_folder)
predicted_labels_list[1]

In [ ]:
# Get list of predicted labels
predicted_labels_list = glob.glob(f"{target_output_folder}*.nii.gz")
# predicted_labels_list = os.listdir(target_output_folder)
# predicted_labels_list_dicts = [{DataDict.FileName: predicted_label_list, DataDict.Label: os.path.join(target_output_folder, predicted_label_list)} for predicted_label_list in predicted_labels_list]
predicted_labels_list_dicts = [{DataDict.FileName: os.path.basename(predicted_label_list), DataDict.Label: predicted_label_list} for predicted_label_list in predicted_labels_list]
correct_prection_list = []
incorrect_microbleed_dict_list = []

target_file_check = None

# Iterate through each file
for image_idx, predicted_label_list_dicts in enumerate(predicted_labels_list_dicts):
    brain_image_filename = predicted_label_list_dicts[DataDict.FileName].replace("_Label", "")
    label_filename = predicted_label_list_dicts[DataDict.FileName]
    
    # Load coordinate systems for current filename
    current_image_microbleeds_dicts = get_dicts_from_dicts(test_data_dicts, DataDict.FileName, brain_image_filename)

    # Append dict properties together
    for current_image_microbleed_dict in current_image_microbleeds_dicts:
        current_image_microbleed_dict[DataDict.Label] = predicted_label_list_dicts[DataDict.Label]

    prediction_correct = 0

    # For each microbleed check if coordinate is more than zero
    for microbleed_idx, microbleed_dict in enumerate(current_image_microbleeds_dicts):
        x = microbleed_dict[DataDict.X]
        y = microbleed_dict[DataDict.Y]
        z = microbleed_dict[DataDict.Z]
        
        # Load predicted label
        label_image = nb.load(microbleed_dict[DataDict.Label]).get_fdata()

        # correct_microbleed_prediction = label_image[x, y, z] > 0
        # Give a +- 2 pixel leeway 
        boundary_slack = 3
        correct_microbleed_prediction = 1 in label_image[x-boundary_slack:x+boundary_slack, y-boundary_slack:y+boundary_slack, z-boundary_slack:z+boundary_slack]

        if correct_microbleed_prediction:
            prediction_correct += 1
        else:
            incorrect_microbleed_dict_list.append({DataDict.FileName: label_filename, "Coordinates": (x, y, z)})
        
    print(f"{image_idx}/{len(predicted_labels_list_dicts)} subject: {prediction_correct}/{len(current_image_microbleeds_dicts)} correctly predicted")
    # correct_prection_list[f"Subject-{image_idx}"] = {"Correct_Predictions": prediction_correct, DataDict.Filename: label_filename}
    correct_prection_list.append({"Correct_Predictions": prediction_correct, DataDict.FileName: label_filename, "Total_Micrbleeds": len(current_image_microbleeds_dicts)})

# current_image_microbleeds_dicts
correct_prection_list

In [ ]:
incorrect_total = 0
total_microbleeds = 0

for correct_prection_list_item in correct_prection_list:
    incorrect_cur = correct_prection_list_item['Total_Micrbleeds']- correct_prection_list_item["Correct_Predictions"]
    incorrect_total += incorrect_cur
    total_microbleeds += correct_prection_list_item['Total_Micrbleeds']

#CHECK THIS

total_microbleeds = len(test_data_dicts)
print(f"Incorrect Total: {incorrect_total}\tMicrobleed Total: {total_microbleeds}\tPercentage Correct: {round((total_microbleeds-incorrect_total)/total_microbleeds*100, 2)}%")

In [ ]:
incorrect_microbleed_dict_list
with open(os.path.join(target_output_folder, 'incorrect_microbleed_dict_list.txt'), 'w') as f:
    f.write(str(incorrect_microbleed_dict_list))

In [ ]:
predicted_labels_list_dicts

In [ ]:
test_load = nb.load(f"{target_output_folder}122_T1_MRI_SWI_BFC_50mm_HM_rsCMB_V2_Label.nii.gz")
test_image = test_load.get_fdata()
print(test_image.shape)
# 140, 116, 28
#118, 196, 31
coord = (118, 196, 31)
x = coord[0]
y = coord[1]
z = coord[2]
plt.imshow(test_image[x-20:x+20, y-20:y+20, z], cmap='gray')

plt.imshow(test_image[..., z], cmap='gray')

In [ ]:
coordinates_test_1 = [(item[DataDict.X], item[DataDict.Y], item[DataDict.Z]) for item in same_subject_microbleeds_dict]
coordinates_test_1

In [ ]:
coord_sample = coordinates_test_1[2]
x = coord_sample[0]
y = coord_sample[1]
z = coord_sample[2]

plt.imshow(tensor_to_cpu(test_label[x-30:x+30, y-30:y+30, z]), cmap="gray")

TO DO: 
- Retrain network as 4.pt
- Get all predicted labels
- Compare against coordinates to find coordinates with center pixel != 1
- manually annotate non annotated microbleeds
- train all data on network. 

In [ ]:
list_axis_errors

In [ ]:
len(patch_error_filenames)

In [ ]:
# patch_error_filenames_unique = [item for item in patch_error_filenames if item ]
patch_error_filenames_unique = np.array(patch_error_filenames)
patch_error_filenames_unique = np.unique(patch_error_filenames_unique)
patch_error_filenames_unique = list(patch_error_filenames_unique)
len(patch_error_filenames_unique)

In [ ]:
os.cwd()